In [123]:
import requests
from bs4 import BeautifulSoup
cookies = {
    'lang': 'en',
    '_gid': 'GA1.2.1458066405.1665327352',
    '_gcl_au': '1.1.1248996888.1665327352',
    'non_customer_id': '6342e0f896625',
    'fuel_csrf_token': 'd359c4331f64bb5935d21bf8cdc97c19e7a2e56156c2fc154beef7e26c8b933b1eae2a57f65017e1a6938bb9bab373dd32ec86c84c1747c63ed91d338a6e99dc',
    '_gat_UA-3285977-2': '1',
    '_ga_GN4G93Y36X': 'GS1.1.1665327353.3.1.1665327610.0.0.0',
    '_ga': 'GA1.1.377355291.1664216774',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    # Requests sorts cookies= alphabetically
    # 'Cookie': 'lang=en; _gid=GA1.2.1458066405.1665327352; _gcl_au=1.1.1248996888.1665327352; non_customer_id=6342e0f896625; fuel_csrf_token=d359c4331f64bb5935d21bf8cdc97c19e7a2e56156c2fc154beef7e26c8b933b1eae2a57f65017e1a6938bb9bab373dd32ec86c84c1747c63ed91d338a6e99dc; _gat_UA-3285977-2=1; _ga_GN4G93Y36X=GS1.1.1665327353.3.1.1665327610.0.0.0; _ga=GA1.1.377355291.1664216774',
    'Referer': 'https://www.pieroth.jp/item',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="105", "Not)A;Brand";v="8", "Chromium";v="105"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

response = requests.get('https://www.pieroth.jp/item', cookies=cookies, headers=headers)

In [124]:
def get_response(page_no):
    params = {
        'page': page_no,
    }
    response_page = requests.get('https://www.pieroth.jp/item', params=params, cookies=cookies, headers=headers)
    return response_page

In [125]:
def get_total_no_pages():
    soup = BeautifulSoup(response.text,'html.parser')
    try:
        #get total item
        items_div = soup.find('div',class_='lg:w-3/4 w-full').find('div',class_= 'w-full').find('div',class_='lg:flex hidden items-center justify-between mb-50px')
        total_items = int(items_div.find('p',class_='font-300 text-lg').get_text(strip=True).replace('ITEMS',''))
        print(total_items)
        # get item per page
        item_list_div = soup.find('div',class_='itemList').find('ul',class_='goodsList')
        list_per_page = int(len(item_list_div))
        print(list_per_page)
        total_page = round(total_items/list_per_page)
    except:pass

    return total_page

In [126]:
from library.img_parser import image_name_parser
from slugify import slugify

def get_product_data(soup):
    main_url = 'https://www.pieroth.jp'
    # response = requests.get('https://www.pieroth.jp/item/detail/1_1_9286410_1/2020', cookies=cookies, headers=headers)
    item = {
        'tax_included' :True
    }
    item_list = soup.find('div',class_='itemList').find('ul',class_='goodsList')
    for value in item_list:
        #item soup
        item_url = value.find('div',class_='text-sm text-left mb-5px').find('a')['href']
        response_url = requests.get(f'{main_url}{item_url}',headers=headers)
        product_soup = BeautifulSoup(response_url.text,'html.parser')
        #item name and url seo name
        try:
            product_url = f"{main_url}{item_url}"
            # name = value.find('div',class_='text-sm text-left mb-5px').find('a').get_text(strip=True)
            product_name = product_soup.find('div',class_='mb-20px lg:px-0 md:px-40px px-30px').find('h2',class_='text-xs text-blue01 font-200 pb-5px').get_text(strip=True)
            item['name'] = product_name
            item['name_en'] = product_name
            item['seo_name'] = slugify(product_name)
            item['product_url'] = product_url
        except:
            pass
        #item image and image bucket
        try:
            item_image = product_soup.find('a',class_= 'detail__slider_img cloud-zoom')['href']
            image_url = f'{main_url}{item_image}'
            image_name = image_url.split('/')[-1].split('?')[0]
            item['image_url'] = [image_url]
            item['image_bucket'] = [image_name_parser(image_name, FOLDER_NAME)]
        except:
            pass
        #sku
        try:
            product_sku = product_soup.find('div',class_= 'flex flex-wrap lg:mb-80px mb-20px').find('div',class_='mb-20px lg:px-0 md:px-40px px-30px').find('p',class_='text-xs text-grey02 font-200 pb-5px').get_text(strip=True)
            item['sku'] = product_sku
            vintage = product_soup.find('div',class_= 'flex flex-wrap lg:mb-80px mb-20px').find('div',class_='mb-20px lg:px-0 md:px-40px px-30px').find('p',class_='text-xs font-200').get_text(strip=True)
            item['vintage'] = vintage
        except:pass
        #winery
        try:
            # product_details = product_soup.find('div',class_= 'class="text-xs mb-20px lg:px-0 md:px-40px px-30px"')
            product_info = product_soup.find_all('div',class_='flex mb-10px')
            for info in product_info:
                print(1)
                table_data = info.find('h3',class_='mr-5px w-120px text-xs text-justify md:mr-10px').get_text(strip=True)
                print(table_data)
                if table_data == '生産者':
                    winery = info.find('div',class_='flex-1').find('ul',class_='flex flex-wrap text-xs').find('li',class_='pr-2px').find('a').get_text(strip=True)
                    print('winery')
                if table_data =='生産地':
                    print('winery area')
                if table_data =='タイプ':
                    print('type')
                if table_data =='葡萄品種':
                    print('grape variant')
                if table_data =='テイスト':
                    print('taste')
                if table_data =='アルコール度数（%）':
                    print('alcohol')
                if table_data =='サーブする温度':
                    print('serving temperature')
            # item['winery']  = product_details
        except:pass
        print(item)


In [127]:
# main
try:
    OUTPUT = []
    FOLDER_NAME = 'pieroth-img'
    total_pages = get_total_no_pages()
    for page in range(1,5):
        page_response = get_response(page)
        page_soup = BeautifulSoup(page_response.text,'html.parser')
        product_data = get_product_data(page_soup)
        print(page)


    print(total_pages + 1)
except Exception as e:
    print(e)

699
30
1
生産者
1
生産地
1
タイプ
1
葡萄品種
1
テイスト
1
アルコール度数（%）
1
サーブする温度
1
容量
{'tax_included': True, 'name': 'Son Mayol Rose', 'name_en': 'Son Mayol Rose', 'seo_name': 'son-mayol-rose', 'product_url': 'https://www.pieroth.jp/item/detail/1_1_9286410_1/2020', 'image_url': ['https://www.pieroth.jp/photo/2022/2022/zz-9286410_2020-1.jpg?1664947809'], 'image_bucket': ['wsp-img/pieroth-img/zz-9286410_2020-1.jpg'], 'sku': '9286410', 'vintage': '2020'}
1
生産者
1
生産地
1
タイプ
1
葡萄品種
1
テイスト
1
アルコール度数（%）
1
サーブする温度
1
容量
1
原産地呼称／等級
{'tax_included': True, 'name': 'Diadema Rosso', 'name_en': 'Diadema Rosso', 'seo_name': 'diadema-rosso', 'product_url': 'https://www.pieroth.jp/item/detail/1_1_9300310_1/2018', 'image_url': ['https://www.pieroth.jp/photo/2022/2022/zz-9300310_2018-1.jpg?1664949953'], 'image_bucket': ['wsp-img/pieroth-img/zz-9300310_2018-1.jpg'], 'sku': '9300310', 'vintage': '2018'}
1
生産者
1
生産地
1
タイプ
1
葡萄品種
1
テイスト
1
アルコール度数（%）
1
サーブする温度
1
容量
{'tax_included': True, 'name': 'Chateau de la Gardine', 'name_en'

KeyboardInterrupt: 